# Simple linear regression

In this exercise you will train a linear regression model via gradient descent in the simplest scenario, i.e. recreating an affine function.

The setup is as follows:
* we are given a set of pairs $(x, y)$, where $x$ represents the feature, and $y$ is the target,
* our hypothesis is $h(x) = ax + b$,
* we will use the dataset consisting of set of pairs to figure out the right values for $a$ and $b$,
* to do so we will optimize the loss function: $J(a,b) = \frac{1}{n}\sum_{i=1}^n (y_i - h(x_i))^2$,
* with the loss function in hand we can improve our guesses iteratively:
    * $a^{t+1} = a^t - \text{step\_size} \cdot \frac{\partial J(a,b)}{\partial a}$,
    * $b^{t+1} = b^t - \text{step\_size} \cdot \frac{\partial J(a,b)}{\partial b}$,
* we can end the process after some predefined number of epochs (or when the changes are no longer meaningful).

Let's start with creating the dataset.

(y - ax - b)^2
(y - ax - b) * (y - ax - b)

y^2 - yax - by - yax + a^2x^2 + abx - by + abx + b^2
y^2 + a^2x^2 + b^2 - 2yax + 2abx - 2 by

da = 2ax^2 - 2xy + 2bx
db = 2b + 2ax - 2y


In [1]:
import random

_a = 0.3
_b = 0.5

f = lambda x: _a * x + _b # ground truth
g = lambda x: f(x) + random.gauss(0, 0.02) # a noisy version of f

In [2]:
n = 50 # number of examples

xs = [random.random() for _ in range(n)] # features
ys = list(map(g, xs)) # targets

ts = list(map(f, xs)) # we don't get to see this

Our goal is to recreate $f$. However, as reality can be harsh (and usually is) we only get to observe $g$. We observe it as a list of pairs $(x,y) \in \text{zip}(xs, ys)$.

Let's plot the data. We will use the `plotly` library to make the plots interactive, which allows for easier inspection of data.

In [3]:
!pip install -q plotly==4.2.1

In [4]:
import plotly.graph_objects as go
import plotly.express as px

fig = px.scatter(x=xs, y=ys)
fig.show()

In [12]:
import numpy as np

def mse_loss(ys, ps):
    assert len(ys) == len(ps)

    ### YOUR CODE BEGINS HERE ###

    ys_ = np.array(ys)
    ps_ = np.array(ps)

    loss = (ys_ - ps_)**2
    loss = np.sum(loss) / len(ps)

    #print(loss, len(ps))

    return loss

    ### YOUR CODE ENDS HERE ###


#mse_loss([1,2,3], [2,3,4])

Please take a while to (roughly) guess the output before executing the cell below.

In [13]:
mse_loss(ys, ts)

Let's now implement the algorithm

Hint: To make sure that you correctly compute the gradients, you can compute them numerically and compare the results.

In [20]:
a = 0. # our initial guess for _a
b = 0. # our initial guess for _b
lr = 0.5 # step size

n_epochs = 40 # number of passes over the training data

def predict(a, b, xs=xs):
    return [a * x + b for x in xs]

def evaluate(a, b, xs=xs, ys=ys):
    return mse_loss(ys, predict(a, b, xs))

losses = [evaluate(a, b)]

### CAN I CODE HERE?
xs_ = np.array(xs)
ys_ = np.array(ys)
###

for i in range(n_epochs):
    #############################
    # TODO: Fill in the details #
    #############################
    ### YOUR CODE BEGINS HERE ###

    #da = 2ax^2 - 2xy + 2bx

    da = np.sum(2 * a * xs_ * xs_ - 2 * xs_ * ys_ + 2 * b * xs_)
    a = a - (1 / len(xs)) * lr * da

    #db = 2b + 2ax - 2y

    db = np.sum(2 * b + 2 * a * xs_ - 2 * ys_)
    b = b - (1 / len(xs)) * lr * db

    ### YOUR CODE ENDS HERE ###

    loss = evaluate(a, b)
    losses.append(loss)

    print(f'Iter: {i:>3} Loss: {loss:8.8f} a: {a:8.5f}, b: {b:8.5f}')

In [21]:
fig = px.line(y=losses, labels={'y':'loss'})
fig.show()

Let's now visually asses how we do on training data

In [22]:
fig = go.Figure()

fig = px.scatter(x=xs, y=ys)
dense_x = np.linspace(np.min(xs), np.max(xs), 100)
fig.add_trace(go.Scatter(x=dense_x, y=predict(a, b, dense_x), name='linear fit', mode='lines'))
fig.add_trace(go.Scatter(x=xs, y=ts, name='y without noise', mode='markers'))

fig.show()

Let's check our implementation vs. the one in sklearn and numpy.


In [24]:
from sklearn.linear_model import LinearRegression
import numpy as np

X = np.array(xs).reshape((len(xs), 1))
regr = LinearRegression()
regr.fit(X, ys) # training

sk_a = float(regr.coef_)
sk_b = regr.intercept_
sk_loss = mse_loss(ys, regr.predict(X))

print(f'Loss: {sk_loss:8.8f} a: {sk_a:8.5f}, b: {sk_b:8.5f}')

In [25]:
z = np.polyfit(x=xs, y=ys, deg=1)
print(z)
f = np.poly1d(z)
print(f)